In [1]:
spark.sql("DROP TABLE IF EXISTS icedsparkcatalog.mydb.smalldataset")

Intitializing Scala interpreter ...

Spark Web UI available at http://bc18817f1332:4043
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1720820315519)
SparkSession available as 'spark'


res0: org.apache.spark.sql.DataFrame = []


In [2]:
 import org.apache.spark.sql._
 import org.apache.spark.sql.types._

import org.apache.spark.sql._
import org.apache.spark.sql.types._


In [3]:
 val csvschema =
   StructType(
     StructField("name", IntegerType, true) ::
     StructField("id", LongType, false) ::
     StructField("nametype", StringType, false) :: 
     StructField("recclass", StringType, false) :: 
     StructField("mass", LongType, false) ::
     StructField("fall", StringType, false) ::
     StructField("year", IntegerType, false) ::
     StructField("reclat", DoubleType, false) ::
     StructField("reclong", DoubleType, false) ::
      StructField("GeoLocation", StringType, false) ::
     Nil)


csvschema: org.apache.spark.sql.types.StructType = StructType(StructField(name,IntegerType,true),StructField(id,LongType,false),StructField(nametype,StringType,false),StructField(recclass,StringType,false),StructField(mass,LongType,false),StructField(fall,StringType,false),StructField(year,IntegerType,false),StructField(reclat,DoubleType,false),StructField(reclong,DoubleType,false),StructField(GeoLocation,StringType,false))


In [4]:
val deleteMultipleLessColsDF =  spark.read.option("multiline", "true").schema(csvschema).csv("./Data/csv/Meteorite_Landings.csv")

deleteMultipleLessColsDF: org.apache.spark.sql.DataFrame = [name: int, id: bigint ... 8 more fields]


In [5]:
deleteMultipleLessColsDF.count()

res1: Long = 45717


In [6]:
val icedsmallTable = deleteMultipleLessColsDF.writeTo("icedsparkcatalog.mydb.smalldataset").using("iceberg").createOrReplace()

icedsmallTable: Unit = ()


In [8]:
spark.sql("ALTER TABLE icedsparkcatalog.mydb.smalldataset SET TBLPROPERTIES ('write.spark.accept-any-schema'='true')")

res3: org.apache.spark.sql.DataFrame = []


In [9]:
val deletedDataDF =  spark.read.option("multiline", "true").csv("./Data/csv/Meteorite_Landings_delmult.csv")

deletedDataDF: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 5 more fields]


In [10]:
val deletedIcedTable = deletedDataDF.writeTo("icedsparkcatalog.mydb.smalldataset").option("mergeSchema","true").append()

deletedIcedTable: Unit = ()


In [11]:
val delMultipleIcedTable = spark.read.format("iceberg").load("icedsparkcatalog.mydb.smalldataset")

delMultipleIcedTable: org.apache.spark.sql.DataFrame = [name: int, id: bigint ... 15 more fields]


In [17]:
delMultipleIcedTable.count()

res9: Long = 91434


In [18]:
delMultipleIcedTable.select("year","fall","nametype").head(10)

res10: Array[org.apache.spark.sql.Row] = Array([null,fall,nametype], [1880,Fell,Valid], [1951,Fell,Valid], [1952,Fell,Valid], [1976,Fell,Valid], [1902,Fell,Valid], [1919,Fell,Valid], [1949,Fell,Valid], [1814,Fell,Valid], [1930,Fell,Valid])


In [13]:
delMultipleIcedTable.select("year","fall","nametype").tail(10)

res5: Array[org.apache.spark.sql.Row] = Array([null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null], [null,null,null])


In [14]:
delMultipleIcedTable.select("year","fall","nametype").filter("fall is null").show()

+----+----+--------+
|year|fall|nametype|
+----+----+--------+
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
|NULL|NULL|    NULL|
+----+----+--------+
only showing top 20 rows



In [15]:
delMultipleIcedTable.select("year","fall","nametype").filter("fall is null").count()

res7: Long = 45717
